In [1]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity


import itertools
#sys.path


import regex as re
import datetime
import math
import time
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [2]:
                                  

####  NEW FILE INCLUDING EARLY CITATIONS OF YOUNG REFERENCES:   ../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit.pkl
                                            
#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))



%time df_merged = pickle.load(open('../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit_and_after_accretion_time.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)




df_merged = df_merged[df_merged['cite_count'] != -1]   # i dont know why, but there are 7 occurrences with value -1



plos_df = df_merged.drop_duplicates(subset=['paper_UT'])
print (plos_df.shape)



df_merged.head()

CPU times: user 4.27 s, sys: 779 ms, total: 5.05 s
Wall time: 5.05 s
done loading pickles (5787634, 34)
(156558, 34)


,occurence,reference_UT,reference_rank,regex_sect_index,cite_count,ref_pub_year,paper_cite_count,plos_pub_year,sect_char_pos,sect_char_total,...,plos_article_type,num_cit_young_ref_by2009,num_cit_young_ref_by2010,num_cit_young_ref_by2011,num_cit_young_ref_by2012,num_cit_young_ref_by2013,num_cit_young_ref_by2009after8,num_cit_young_ref_by2008after8,num_cit_young_ref_by2010after8,num_cit_young_ref_by2007after8
0,1,A1995QY75100004,1,0,60.0,1995.0,2,2013.0,139,4029,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A1995QY75100004,1,3,60.0,1995.0,2,2013.0,494,5398,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,000263911400006,3,0,5.0,2009.0,2,2013.0,142,4029,...,@ Article,0.0,0.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0
5,1,000289279600018,4,0,29.0,2011.0,2,2013.0,269,4029,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0
6,3,000289279600018,4,3,29.0,2011.0,2,2013.0,3844,5398,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0


In [4]:
######  i create the new columns for fraction of references in each section (labelled 0 to 7)



%time plos_df = pickle.load(open('../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect_young_old.pkl', 'rb'))
print ("done loading plos_df", plos_df.shape)
plos_df_simplified = plos_df[['paper_UT','num_ref_section0','num_ref_section1','num_ref_section2','num_ref_section3','num_ref_section4','num_ref_section5','num_ref_section6','num_ref_section7',
    'fract_old_ref_section0', 'fract_old_ref_section1', 'fract_old_ref_section2', 'fract_old_ref_section3', 'fract_old_ref_section4', 'fract_old_ref_section5', 'fract_old_ref_section6', 'fract_old_ref_section7',\
    'fract_young_ref_section0', 'fract_young_ref_section1', 'fract_young_ref_section2', 'fract_young_ref_section3', 'fract_young_ref_section4', 'fract_young_ref_section5', 'fract_young_ref_section6', 'fract_young_ref_section7']]

# print ("plos_df_simplified", plos_df_simplified.shape)




#df_merged = pd.merge(df_merged, plos_df_simplified, on='paper_UT', how='left')
df_merged = pd.merge(df_merged, plos_df_simplified, on='paper_UT', how='left')
print ("df_merged", df_merged.shape)

# df_merged[['paper_UT','reference_UT','regex_sect_index','eff_num_ref','num_ref_section0','num_ref_section1','num_ref_section2','num_ref_section3','num_ref_section4','num_ref_section5','num_ref_section6','num_ref_section7']].head(100)




df_records_one_ref_per_plos = df_merged.drop_duplicates(subset=['paper_UT'])
print (df_records_one_ref_per_plos.shape)



# df_records_one_ref_per_plos.rename(columns={'paper_cite_count_x': 'paper_cite_count', 'plos_pub_year_x': 'plos_pub_year', 'ref_pub_year_x':'ref_pub_year','regex_sect_index_x':'regex_sect_index', 'total_refs_x':'total_refs'}, inplace=True)
# df_merged.rename(columns={'paper_cite_count_x': 'paper_cite_count', 'plos_pub_year_x': 'plos_pub_year', 'ref_pub_year_x':'ref_pub_year','regex_sect_index_x':'regex_sect_index', 'total_refs_x':'total_refs','reference_UT_x':'reference_UT','cite_count_x':'cite_count'}, inplace=True)


sorted(df_records_one_ref_per_plos.columns)

df_records_one_ref_per_plos.head()



###OJO!!!!! FALTA POR HACER BIEN EL MERGE PARA EVITAR REPETICION DE COLUMNAS!!!

CPU times: user 442 ms, sys: 88.4 ms, total: 530 ms
Wall time: 529 ms
done loading plos_df (156610, 87)
df_merged (5787630, 58)
(156558, 58)


,occurence,reference_UT,reference_rank,regex_sect_index,cite_count,ref_pub_year,paper_cite_count,plos_pub_year,sect_char_pos,sect_char_total,...,fract_old_ref_section6,fract_old_ref_section7,fract_young_ref_section0,fract_young_ref_section1,fract_young_ref_section2,fract_young_ref_section3,fract_young_ref_section4,fract_young_ref_section5,fract_young_ref_section6,fract_young_ref_section7
0,1,A1995QY75100004,1,0,60.0,1995.0,2,2013.0,139,4029,...,0.0,0.0,0.000000,0.0,0.0,0.032258,0.0,0.0,0.0,0.0
31,1,000088374400026,1,0,477.0,2000.0,2,2015.0,543,2207,...,0.0,0.0,0.023256,0.0,0.0,0.023256,0.0,0.0,0.0,0.0
74,1,000313346100007,2,0,6.0,2013.0,1,2015.0,591,2474,...,0.0,0.0,0.181818,0.0,0.0,0.090909,0.0,0.0,0.0,0.0
85,1,000301376000017,2,0,35.0,2012.0,8,2012.0,762,2700,...,0.0,0.0,0.130435,0.0,0.0,0.086957,0.0,0.0,0.0,0.0
108,1,000232311300030,1,0,1687.0,2005.0,98,2008.0,77,3195,...,0.0,0.0,0.000000,0.0,0.0,0.040000,0.0,0.0,0.0,0.0


In [5]:
### FIGURE 2C, HEATMAP FOR NUMBER OF YOUNG  REFERENCES




list_q=[0.3,0.6,.9,.99,1]    
    
#### i get the bins number of citation of the plos papers OJO!!!!! i want the same bins for all papers (so i calculate them before separating into sections but after all the preselections)

#quantiles=sorted(list(df_plos[string_filtering_x].quantile(list_q).to_dict().items())) #mean 10.68 
quantiles=sorted(list(df_records_one_ref_per_plos['paper_cite_count'].quantile(list_q).to_dict().items())) #mean 10.68 

lista_bins_plos_citations=[]
old_value=0
for item in quantiles:
    try:
        pair=[old_value, int(item[1])]    
    except:  # if it is a nan:
        pair=[old_value, item[1]]
    
    lista_bins_plos_citations.append(pair)
    
    try:
        old_value = int(item[1])
    except:
        old_value = item[1]

print (lista_bins_plos_citations, df_records_one_ref_per_plos.shape)


lista_text_z = []
list_of_lists =[]
for pair_values in lista_bins_plos_citations:       
    
    minimo = pair_values[0]
    maximo = pair_values[1]   

    df_select = df_records_one_ref_per_plos[(df_records_one_ref_per_plos['paper_cite_count'] >= minimo)  &  (df_records_one_ref_per_plos['paper_cite_count'] < maximo)]            
    
    list_q_cell = [.25,.5,.75]
    list_quantiles = []   # for [intro, methods, results, discussion]
    for column in ['fract_young_ref_section0', 'fract_young_ref_section1', 'fract_young_ref_section2', 'fract_young_ref_section3']:
        quantiles_cell_aux  = sorted(list(df_select[column].quantile(list_q_cell).to_dict().items())) #mean 10.68 
        list_quantiles.append(quantiles_cell_aux)
    print (pair_values, df_records_one_ref_per_plos.shape, df_select.shape, list_quantiles)
   


    #example   list_quantiles: [[(0.25, 7.0), (0.5, 12.0), (0.75, 17.0)],      [(0.25, 1.0), (0.5, 3.0), (0.75, 6.0)],      [(0.25, 0.0), (0.5, 0.0), (0.75, 2.0)],      [(0.25, 5.0), (0.5, 11.0), (0.75, 18.0)]]
    
    
  
    
    #aux_list = [df_select.fract_young_ref_section0.mean(), df_select.fract_young_ref_section1.mean(), df_select.fract_young_ref_section2.mean(), df_select.fract_young_ref_section3.mean() ]      # [intro, methods, results, discussion]
    aux_list = [df_select.fract_young_ref_section0.mean()*100., df_select.fract_young_ref_section1.mean()*100., df_select.fract_young_ref_section2.mean()*100., df_select.fract_young_ref_section3.mean()*100. ]      # [intro, methods, results, discussion]
    
    
    list_of_lists.append(aux_list)
    
    
    
    aux_z  = [ round(i,2) for i in aux_list]
    lista_text_z.append(aux_z)
    
    
    
    
    
#     lista_trios = []
#     for lista_pares in list_quantiles:
#         trio_valores = [ int(i[1]) for i in lista_pares]
#         lista_trios.append(trio_valores)
    
#     print (aux_list,"\n", lista_trios,'\n')
    
    
    
   
     #aux_lista_text_z =  [str(lista_trios[i]).replace('[','').replace(']','').replace(', ', '-')   + ""     for i in range(len(aux_list))]     # [intro, methods, results, discussion]
                                                                                                                                       
                                                                                                                                          
                                                                                                                                          
#     aux_lista_text_z =  [str(lista_trios[i]).replace('[','').replace(']','').replace(', ', '-')   + ""     for i in range(len(aux_list))]     # [intro, methods, results, discussion]
  
#     lista_text_z.append(aux_lista_text_z)
    
# print ("\n\n")
# for lista in list_of_lists:
#     print (lista)

# print ("\n\n")
# for lista in lista_text_z:
#     print (lista)
 










# fig_colorscale = [[0, '#dcf0d2'], [1, '#205803']]   # if i give it a min and a max colors in HEX, it creates a gradient from one to another
# fig_font_colors = ['#205803', '#dcf0d2']      # same for the annotation of the boxes (to make sure they are readable)
# fig_title_plot = "Median difference between publication year of plos and references in "+string_journal+string_plos_field+" papers from "+str(years)+string_isolated_ref+string_self_ref+string_code_categ+"<br>Number of occurrences: "+str(N_all)#+",   Number plos: "+str(N_plos)
# fig_filename = '../plots/annotated-heatmap_median_age_difference_plos_publ_year_vs_references_for_sections_and_subsect_by_citations_of_plos'





colorbar_string = 'Percentage young references'     #Relative fraction of References'


lista_bin_names=["Bottom","Typical","Good","High","Top"]
lista_sections = ["Introduction","Methods","Results","Discussion"]





#fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names, annotation_text=lista_text_z, colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True, colorbar=dict(title=colorbar_string, titleside='right' ),)#, reversescale=True)


###  i round up the values for the labels of each cell to one single decimal
print (lista_text_z)
for i in range(len(lista_text_z)):
    item_lista= lista_text_z[i]
    for j in range(len(item_lista)):
        item = item_lista[j]
        lista_text_z[i][j] =  round(item,1) 

print ("\n\n")
print (lista_text_z)


    
fig = ff.create_annotated_heatmap(z=list_of_lists, x=lista_sections, y=lista_bin_names, annotation_text = lista_text_z,  colorscale='Viridis', showscale=True,\
                                  colorbar=dict(title=colorbar_string,titleside='right',ticks='outside' )) #colorbar=dict(title=colorbar_string,x=.0,y=.95, titleside='right' ))










#fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names,  colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True)#, reversescale=True)
#fig.layout.title = 'Percentage Young references'

fig['layout']['xaxis']['side'] = 'bottom'
fig.layout.xaxis.update({'title': 'Section'})


fig.layout.yaxis.update({'title': 'Impact Group'})


         
         


font_gral=55  # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral


# Altering x axis7
#fig['layout']['xaxis']['tickfont']['family'] = 'Gill Sans MT'
fig['layout']['xaxis']['tickangle'] = 0
fig['layout']['yaxis']['tickangle'] = -90
fig['layout']['xaxis']['titlefont']['size'] = font_gral  +15
fig['layout']['yaxis']['titlefont']['size'] = font_gral

fig['layout']['xaxis']['tickfont']['size'] = font_gral -7
fig['layout']['yaxis']['tickfont']['size'] = font_gral -15







fig['layout']['margin']=dict(
        l=200,
       # r=50,
        b=150,
        t=150,
        pad=15
    )



# fig.layout.update({      
#      'annotations' : [
         
#          dict(  # this is for the xaxis label
#             x=0.,
#             y=1.05,
#             xref='paper',
#             yref='paper',
#             text="(c)",
#             showarrow=False,  
#             font=dict(               
#                 size=45,),
#            ),  ]  
#     })
         
         



offline.plot(fig, auto_open=True, image = 'png', image_filename='testing_annotated_heatmap_color' ,image_width=2000, image_height=1200, filename='testing_annotated_heatmap_color.html', validate=True)



[[0, 2], [2, 8], [8, 25], [25, 79], [79, 1173]] (156558, 58)
[0, 2] (156558, 58) (35805, 58) [[(0.25, 0.0), (0.5, 0.0), (0.75, 0.038461538461538464)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.04)]]
[2, 8] (156558, 58) (57980, 58) [[(0.25, 0.0), (0.5, 0.0), (0.75, 0.0425531914893617)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0)], [(0.25, 0.0), (0.5, 0.009523809523809525), (0.75, 0.047619047619047616)]]
[8, 25] (156558, 58) (46718, 58) [[(0.25, 0.0), (0.5, 0.0196078431372549), (0.75, 0.05063291139240506)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0)], [(0.25, 0.0), (0.5, 0.024390243902439025), (0.75, 0.06)]]
[25, 79] (156558, 58) (14457, 58) [[(0.25, 0.0), (0.5, 0.02857142857142857), (0.75, 0.06451612903225806)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.015625)], [(0.25, 0.0), (0.5, 0.03508771929824561), (0.75, 0.

'file:///home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/notebooks/testing_annotated_heatmap_color.html'

In [6]:
### FIGURE 2D, HEATMAP FOR NUMBER OF OLD  REFERENCES




list_q=[0.3,0.6,.9,.99,1]    
    
#### i get the bins number of citation of the plos papers OJO!!!!! i want the same bins for all papers (so i calculate them before separating into sections but after all the preselections)

#quantiles=sorted(list(df_plos[string_filtering_x].quantile(list_q).to_dict().items())) #mean 10.68 
quantiles=sorted(list(df_records_one_ref_per_plos['paper_cite_count'].quantile(list_q).to_dict().items())) #mean 10.68 

lista_bins_plos_citations=[]
old_value=0
for item in quantiles:
    try:
        pair=[old_value, int(item[1])]    
    except:  # if it is a nan:
        pair=[old_value, item[1]]
    
    lista_bins_plos_citations.append(pair)
    
    try:
        old_value = int(item[1])
    except:
        old_value = item[1]

print (lista_bins_plos_citations, df_records_one_ref_per_plos.shape)


lista_text_z = []
list_of_lists =[]
for pair_values in lista_bins_plos_citations:       
    
    minimo = pair_values[0]
    maximo = pair_values[1]   

    df_select = df_records_one_ref_per_plos[(df_records_one_ref_per_plos['paper_cite_count'] >= minimo)  &  (df_records_one_ref_per_plos['paper_cite_count'] < maximo)]            
    
    list_q_cell = [.25,.5,.75]
    list_quantiles = []   # for [intro, methods, results, discussion]
    for column in ['fract_old_ref_section0', 'fract_old_ref_section1', 'fract_old_ref_section2', 'fract_old_ref_section3']:
        quantiles_cell_aux  = sorted(list(df_select[column].quantile(list_q_cell).to_dict().items())) #mean 10.68 
        list_quantiles.append(quantiles_cell_aux)
    print (pair_values, df_records_one_ref_per_plos.shape, df_select.shape, list_quantiles)
   

   
    
  
    
    #aux_list = [df_select.fract_old_ref_section0.mean(), df_select.fract_old_ref_section1.mean(), df_select.fract_old_ref_section2.mean(), df_select.fract_old_ref_section3.mean() ]      # [intro, methods, results, discussion]
    aux_list = [df_select.fract_old_ref_section0.mean()*100., df_select.fract_old_ref_section1.mean()*100., df_select.fract_old_ref_section2.mean()*100., df_select.fract_old_ref_section3.mean()*100. ]      # [intro, methods, results, discussion]
    
    
    list_of_lists.append(aux_list)
    
    
    
    aux_z  = [ round(i,2) for i in aux_list]
    lista_text_z.append(aux_z)
    
    
    
    


colorbar_string = 'Percentage old references'     #Relative fraction of References'


lista_bin_names=["Bottom","Typical","Good","High","Top"]
lista_sections = ["Introduction","Methods","Results","Discussion"]





#fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names, annotation_text=lista_text_z, colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True, colorbar=dict(title=colorbar_string, titleside='right' ),)#, reversescale=True)


###  i round up the values for the labels of each cell to one single decimal
print (lista_text_z)
for i in range(len(lista_text_z)):
    item_lista= lista_text_z[i]
    for j in range(len(item_lista)):
        item = item_lista[j]
        lista_text_z[i][j] =  round(item,1) 

print ("\n\n")
print (lista_text_z)



    
fig = ff.create_annotated_heatmap(z=list_of_lists, x=lista_sections, y=lista_bin_names, annotation_text = lista_text_z,  colorscale='Viridis', showscale=True,\
                                  colorbar=dict(title=colorbar_string,titleside='right',ticks='outside' )) #colorbar=dict(title=colorbar_string,x=.0,y=.95, titleside='right' ))











#fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names,  colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True)#, reversescale=True)
#fig.layout.title = 'Old references'

fig['layout']['xaxis']['side'] = 'bottom'
fig.layout.xaxis.update({'title': 'Section'})


fig.layout.yaxis.update({'title': 'Impact Group'})


 



font_gral=55  # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral


# Altering x axis7
#fig['layout']['xaxis']['tickfont']['family'] = 'Gill Sans MT'
fig['layout']['xaxis']['tickangle'] = 0
fig['layout']['yaxis']['tickangle'] = -90
fig['layout']['xaxis']['titlefont']['size'] = font_gral  +15
fig['layout']['yaxis']['titlefont']['size'] = font_gral

fig['layout']['xaxis']['tickfont']['size'] = font_gral -7
fig['layout']['yaxis']['tickfont']['size'] = font_gral -15


fig['layout']['margin']=dict(
        l=200,
       # r=50,
        b=150,
        t=150,
        pad=15
    )






offline.plot(fig, auto_open=True, image = 'png', image_filename='testing_annotated_heatmap_color' ,image_width=2000, image_height=1200, filename='testing_annotated_heatmap_color.html', validate=True)



[[0, 2], [2, 8], [8, 25], [25, 79], [79, 1173]] (156558, 58)
[0, 2] (156558, 58) (35805, 58) [[(0.25, 0.07407407407407407), (0.5, 0.14705882352941177), (0.75, 0.23809523809523808)], [(0.25, 0.0), (0.5, 0.04395604395604396), (0.75, 0.0975609756097561)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.02857142857142857)], [(0.25, 0.029411764705882353), (0.5, 0.11363636363636363), (0.75, 0.20930232558139536)]]
[2, 8] (156558, 58) (57980, 58) [[(0.25, 0.06521739130434782), (0.5, 0.13043478260869565), (0.75, 0.2127659574468085)], [(0.25, 0.0), (0.5, 0.04081632653061224), (0.75, 0.08695652173913043)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.0425531914893617)], [(0.25, 0.03333333333333333), (0.5, 0.10714285714285714), (0.75, 0.1935483870967742)]]
[8, 25] (156558, 58) (46718, 58) [[(0.25, 0.04838709677419355), (0.5, 0.10416666666666667), (0.75, 0.17777777777777778)], [(0.25, 0.0), (0.5, 0.03571428571428571), (0.75, 0.07692307692307693)], [(0.25, 0.0), (0.5, 0.0), (0.75, 0.04878048780487805)], [(0.25, 0.02564102

'file:///home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/notebooks/testing_annotated_heatmap_color.html'